# <span style="color:blue">Pre-analysis to find the optimal number of topics and hyperparameters values (2019_Feb)</span>

In [1]:
yyyy_MMM = '2019_Feb'

### Load packages

In [2]:
import csv, os, re, time
import numpy as np
import pandas as  pd
import matplotlib.pyplot as plt  
import pyLDAvis
import pyLDAvis.gensim
import gensim
import gensim.corpora as corpora
from gensim.corpora.dictionary import Dictionary
from gensim.models import CoherenceModel
from gensim.models import Phrases
from gensim.test.utils import datapath
from gensim.utils import simple_preprocess
from pprint import pprint
import nltk
nltk.download('stopwords')

C:\Users\minio\anaconda3\envs\tfm-py-37\lib\site-packages\botocore\vendored\requests\packages\urllib3\_collections.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping, MutableMapping
C:\Users\minio\anaconda3\envs\tfm-py-37\lib\site-packages\botocore\vendored\requests\packages\urllib3\_collections.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping, MutableMapping
C:\Users\minio\anaconda3\envs\tfm-py-37\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
C:\Users\minio\anaconda3\envs\tfm-py-37\lib\site-packages\gensim\models\doc

True

### Constants

In [3]:
# for reproducible results
random_state = 33
# if debug is True, messages during the process are generated
debug = False
# for debugging purposes sample may be used to work with a smaller amount of data (when 0 it is not applied)
sample = 0
# if the test is True, the process runs on a smaller subset of raw data (json files)
test = False

if test:
    # path where the doc files are stored
    path_doc = r'D:\master\data science\semestre 4\M2.979 - tfm\data\bins_test\doc'
    # path where the model files are stored
    path_model = r'D:\master\data science\semestre 4\M2.979 - tfm\data\bins_test\model_hyperparameter'
    # LDA chunksize
    chunksize = 250
else:
    # path where the doc files are stored
    path_doc = r'D:\master\data science\semestre 4\M2.979 - tfm\data\bins\doc'
    # path where the model files are stored
    path_model = r'D:\master\data science\semestre 4\M2.979 - tfm\data\bins\model_hyperparameter'
    # LDA chunksize
    chunksize = 5000

file_doc = os.path.join(path_doc, yyyy_MMM, yyyy_MMM) + '_28_results.csv'
file_model = os.path.join(path_model, yyyy_MMM, 'model')
print(file_doc)
print(file_model)

D:\master\data science\semestre 4\M2.979 - tfm\data\bins\doc\2019_Feb\2019_Feb_28_results.csv
D:\master\data science\semestre 4\M2.979 - tfm\data\bins\model_hyperparameter\2019_Feb\model


C:\Users\minio\anaconda3\envs\tfm-py-37\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Load data preprocessed

In [4]:
df = pd.read_csv(file_doc)
documents = df.content_preprocessed
documents.head(10)

C:\Users\minio\anaconda3\envs\tfm-py-37\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0    ['steppin', 'season', 'hair', 'attitude', 'eee...
1    ['comfortable', 'express', 'boundary', 'uncomf...
2                         ['random', 'fabric', 'fold']
3    ['aiya', 'veggie', 'harder', 'expect', 'live',...
4                                          ['clothes']
5                     ['throwback', 'omega', 'choice']
6                                ['everyday', 'cheat']
7    ['island', 'hour', 'place', 'guess', 'healthy'...
8    ['post', 'workout', 'video', 'record', 'year',...
9               ['finally', 'visit', 'nova', 'scotia']
Name: content_preprocessed, dtype: object

### Shuffle

In [5]:
documents = documents.sample(frac=1, random_state=random_state)
documents.head(10)

C:\Users\minio\anaconda3\envs\tfm-py-37\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


76615    ['fourth', 'strong', 'cafe', 'open', 'door', '...
85882    ['forward', 'summer', 'chicken', 'fulk', 'desi...
85773    ['initial', 'clean', 'clear', 'design', 'leath...
31374    ['beauty', 'cosmetic', 'green', 'dragon', 'leg...
54501    ['panas', 'bookshop', 'fauve', 'dream', 'virgi...
18776    ['find', 'anna', 'karin', 'karlsson', 'collect...
91211    ['hola', 'chicas', 'hola', 'ibiza', 'good', 'v...
83732    ['maternity', 'dress', 'peony', 'pregnant', 't...
26964    ['anniversary', 'vogue', 'talent', 'vogue', 't...
7364     ['diary', 'beauty', 'addict', 'food', 'food', ...
Name: content_preprocessed, dtype: object

### Sample

In [6]:
if sample > 0:
    documents = documents.sample(n=sample, random_state=random_state)
    print(documents.head(10))

C:\Users\minio\anaconda3\envs\tfm-py-37\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Tokenize words and Clean-up text  
https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#3importpackages

In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

C:\Users\minio\anaconda3\envs\tfm-py-37\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Function to compute bigrams & trigrams  
https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html#pre-process-and-vectorize-the-documents

In [8]:
# Add bigrams and trigrams to docs (only ones that appear min_count times or more).
def bi_tri_grams(documents, min_count):
    bigram = Phrases(documents, min_count=min_count)
    for idx in range(len(documents)):
        for token in bigram[documents[idx]]:
            if '_' in token:
                # Token is a bigram, add to document.
                documents[idx].append(token)

C:\Users\minio\anaconda3\envs\tfm-py-37\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Function for categorizing and tagging words

In [9]:
def tag_words(text):
    tags = nltk.pos_tag(text)
    nouns = [word for word, pos in tags if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS')]
    return nouns

C:\Users\minio\anaconda3\envs\tfm-py-37\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Analysis to get the optimal number of topics  
https://towardsdatascience.com/discover-what-singaporeans-generally-talk-on-sms-d4b592a682fd

In [10]:
data_words = list(sent_to_words(documents))
# Compute bigrams & trigrams
bi_tri_grams(data_words, 250)
# Categorizing and tagging words
data_words = list(map(tag_words, data_words))
# Create Dictionary
id2word = Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

C:\Users\minio\anaconda3\envs\tfm-py-37\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
model_list = []
coherence_score = []
model_topics = []

for num_topics in range(8, 12, 1):
    
    time_start = time.time()

    lda_x = gensim.models.LdaMulticore(corpus=corpus
                                       ,id2word=id2word
                                       ,num_topics=num_topics
                                       ,random_state=random_state
                                       ,chunksize=chunksize
                                       ,passes=10)
    coherence_lda_model = CoherenceModel(model=lda_x, texts=data_words, dictionary=id2word, coherence='c_v')
    coherence = coherence_lda_model.get_coherence()
    coherence_score.append(coherence)
    model_topics.append(num_topics)
    model_list.append(lda_x)
    
    print("# Topics: " + str(num_topics) + " Score: " + str(coherence))
    
    time_end = time.time()
    hour, rem = divmod(time_end - time_start, 3600)
    minute, second = divmod(rem, 60)
    print('time elapsed - {:0>2}:{:0>2}:{:05.2f}'.format(int(hour), int(minute), second))

C:\Users\minio\anaconda3\envs\tfm-py-37\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Topics: 8 Score: 0.5053786045826911
time elapsed - 00:04:23.74
# Topics: 9 Score: 0.5259571435897294
time elapsed - 00:04:49.84
# Topics: 10 Score: 0.5205753088576273
time elapsed - 00:04:51.70
# Topics: 11 Score: 0.5132523449731102
time elapsed - 00:04:58.87


C:\Users\minio\anaconda3\envs\tfm-py-37\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
limit=12; start=8; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_score)
plt.xlabel("Number of topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

### Hyperparameter tuning  

https://medium.com/analytics-vidhya/analyzing-amazon-tv-reviews-with-latent-dirichlet-allocation-97d19c3bb878  
https://github.com/Georgebob256/Machine-learning-with-Python/blob/master/Amazon%20TV%20reviews..ipynb

In [13]:
# LDA model and its corresponding Coherence Score
def compute_coherence_values(bow_corpus, id2word, n, a, b):
    lda_model = gensim.models.LdaMulticore(corpus=bow_corpus
                                           ,id2word=id2word
                                           ,num_topics=n
                                           ,random_state=random_state
                                           ,chunksize=chunksize
                                           ,passes=10
                                           ,alpha=a
                                           ,eta=b)
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')
    return coherence_model_lda.get_coherence()

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 8
max_topics = 12
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = [0.01, 0.31, 0.61, 0.91, 'symmetric', 'asymmetric']

# Beta parameter
beta = [0.01, 0.31, 0.61, 0.91, 'symmetric']

model_results = {'Topics': []
                 ,'Alpha': []
                 ,'Beta': []
                 ,'Coherence': []
                }

# iterate through number of topics
iter = 0
for k in topics_range:
    # iterate through alpha values
    for a in alpha:
        # iterare through beta values
        for b in beta:
            iter += 1
            # get the coherence score for the given parameters
            cv = compute_coherence_values(bow_corpus=corpus
                                          ,id2word=id2word
                                          ,n=k
                                          ,a=a
                                          ,b=b)
            # Save the model results
            model_results['Topics'].append(k)
            model_results['Alpha'].append(a)
            model_results['Beta'].append(b)
            model_results['Coherence'].append(cv)
            print('Iteration num - ', iter, ' Topics - ', k, ' Alpha - ', a, ' Beta - ', b, ' Coherence - ', cv)

pd.DataFrame(model_results).to_csv(file_model + '_lda_tuning_results.csv', index=False)

C:\Users\minio\anaconda3\envs\tfm-py-37\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Iteration num -  1  Topics -  8  Alpha -  0.01  Beta -  0.01  Coherence -  0.5118314319114352
Iteration num -  2  Topics -  8  Alpha -  0.01  Beta -  0.31  Coherence -  0.5101637159228043
Iteration num -  3  Topics -  8  Alpha -  0.01  Beta -  0.61  Coherence -  0.5106773646087
Iteration num -  4  Topics -  8  Alpha -  0.01  Beta -  0.91  Coherence -  0.505178682033726
Iteration num -  5  Topics -  8  Alpha -  0.01  Beta -  symmetric  Coherence -  0.5045608299645946
Iteration num -  6  Topics -  8  Alpha -  0.31  Beta -  0.01  Coherence -  0.516538312604259
Iteration num -  7  Topics -  8  Alpha -  0.31  Beta -  0.31  Coherence -  0.5213026218007858
Iteration num -  8  Topics -  8  Alpha -  0.31  Beta -  0.61  Coherence -  0.514435169639383
Iteration num -  9  Topics -  8  Alpha -  0.31  Beta -  0.91  Coherence -  0.5140010907180992
Iteration num -  10  Topics -  8  Alpha -  0.31  Beta -  symmetric  Coherence -  0.5097410496257981
Iteration num -  11  Topics -  8  Alpha -  0.61  Beta -

Iteration num -  86  Topics -  10  Alpha -  asymmetric  Beta -  0.01  Coherence -  0.5439752409740977
Iteration num -  87  Topics -  10  Alpha -  asymmetric  Beta -  0.31  Coherence -  0.556630750600901
Iteration num -  88  Topics -  10  Alpha -  asymmetric  Beta -  0.61  Coherence -  0.5466317848157389
Iteration num -  89  Topics -  10  Alpha -  asymmetric  Beta -  0.91  Coherence -  0.549396180726367
Iteration num -  90  Topics -  10  Alpha -  asymmetric  Beta -  symmetric  Coherence -  0.5473116924016315
Iteration num -  91  Topics -  11  Alpha -  0.01  Beta -  0.01  Coherence -  0.5267444926310698
Iteration num -  92  Topics -  11  Alpha -  0.01  Beta -  0.31  Coherence -  0.5141143019112879
Iteration num -  93  Topics -  11  Alpha -  0.01  Beta -  0.61  Coherence -  0.4953906212613992
Iteration num -  94  Topics -  11  Alpha -  0.01  Beta -  0.91  Coherence -  0.49555504682795254
Iteration num -  95  Topics -  11  Alpha -  0.01  Beta -  symmetric  Coherence -  0.5199060432272241
I

In [14]:
pd.DataFrame(model_results).sort_values(by=['Coherence'], ascending=False)

C:\Users\minio\anaconda3\envs\tfm-py-37\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Topics,Alpha,Beta,Coherence
119,11,asymmetric,symmetric,0.567075
28,8,asymmetric,0.91,0.564638
58,9,asymmetric,0.91,0.562594
27,8,asymmetric,0.61,0.561631
115,11,asymmetric,0.01,0.560236
...,...,...,...,...
113,11,symmetric,0.91,0.492700
112,11,symmetric,0.61,0.492478
98,11,0.31,0.91,0.491798
83,10,symmetric,0.91,0.481125
